# Blackboard


## Libraries

In [1]:
from os.path import dirname, join

import numpy as np
import pandas as pd
import requests
import logging
import sys
import math


from bokeh.io import show, output_notebook, curdoc
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Div, Select, Slider, TextInput, AutocompleteInput, Button, Jitter, WheelZoomTool, BoxZoomTool, ResetTool, PanTool, TapTool, NumeralTickFormatter
from bokeh.plotting import figure
from bokeh.transform import cumsum

from io import StringIO
from datetime import datetime

/Users/julian.vicens/.envs/ub/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


### Logger


In [2]:
# Create a logger
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

# Create a handler that writes log messages to stdout
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)

# Create a formatter and add it to the handler
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# Add the handler to the logger
# logger.addHandler(handler)

# Test the logger
logger.debug("This is a debug")
logger.info("This is a info")
logger.warning("This is a warning")
logger.error("This is an error")

This is a warning
This is an error


### Data from API

Loading CSV data from an API with pagination support. The `load_csv_from_api` function allows for efficient retrieval of large datasets by fetching data in chunks and supporting a maximum row limit. It handles pagination, combines the data into a single DataFrame, and saves the result as a CSV file.

In [3]:
def load_csv_from_api(url, chunk_size=1000, max_rows=None):
    """
    Loads CSV data from an API with pagination support.

    This function retrieves data from the specified API URL in chunks, combines them
    into a single DataFrame, and optionally limits the total number of rows fetched.
    The resulting dataset is saved as a CSV file.

    Args:
        url (str): The base URL of the API endpoint.
        chunk_size (int, optional): The number of rows to fetch in each API call. Defaults to 1000.
        max_rows (int, optional): The maximum number of rows to fetch in total. Defaults to None (no limit).

    Returns:
        pandas.DataFrame: The complete dataset fetched from the API.

    Raises:
        requests.RequestException: If there's an error in making the API request.
    """
    offset = 0
    all_data = []
    total_rows_fetched = 0
    
    logger.info(f"Starting data retrieval from {url}")
    logger.info(f"Chunk size: {chunk_size}, Max rows: {max_rows if max_rows else 'No limit'}")
    
    while True:
        # Construct the URL with offset and limit parameters
        paginated_url = f"{url}?$offset={offset}&$limit={chunk_size}"
        
        # Make the API request
        response = requests.get(paginated_url)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Convert the response content to a pandas DataFrame
            chunk = pd.read_csv(StringIO(response.text))
            
            # If the chunk is empty, we've reached the end of the data
            if chunk.empty:
                logger.info("Received empty chunk. Ending pagination.")
                break
            
            # Append the chunk to our list of DataFrames
            all_data.append(chunk)
            
            # Update total rows fetched
            total_rows_fetched += len(chunk)
            logger.info(f"Fetched {len(chunk)} rows. Total rows so far: {total_rows_fetched}")
            
            # Increment the offset for the next request
            offset += chunk_size
            
            # If we've reached the maximum number of rows, stop
            if max_rows and total_rows_fetched >= max_rows:
                logger.info(f"Reached or exceeded max rows ({max_rows}). Stopping pagination.")
                break
        else:
            logger.error(f"Error fetching data: HTTP {response.status_code}")
            break
    
    # Combine all chunks into a single DataFrame
    logger.info("Combining all fetched data into a single DataFrame")
    full_dataset = pd.concat(all_data, ignore_index=True)
    
    # If max_rows was specified, trim the dataset
    if max_rows and len(full_dataset) > max_rows:
        logger.info(f"Trimming dataset to {max_rows} rows")
        full_dataset = full_dataset.head(max_rows)
    
    logger.info(f"Final dataset size: {len(full_dataset)} rows")
    
    # Save the dataset as a CSV file
    current_date = datetime.now().strftime("%Y%m%d")
    dirname = ''
    filename = f"Estadistica_places_{current_date}.csv"
    full_dataset.to_csv(dirname + filename, index=False)
    logger.info(f"Dataset saved to {filename}")    
    
    return full_dataset

In [12]:
def load_json_from_api(url, chunk_size=1000, format='json', filters=None, max_rows=None):
    """
    Loads JSON data from an API with pagination support.

    This function retrieves data from the specified API URL in chunks, combines them
    into a single DataFrame, and optionally limits the total number of rows fetched.
    The resulting dataset is saved as a CSV file.

    Args:
        url (str): The base URL of the API endpoint.
        chunk_size (int, optional): The number of rows to fetch in each API call. Defaults to 1000.
        max_rows (int, optional): The maximum number of rows to fetch in total. Defaults to None (no limit).

    Returns:
        pandas.DataFrame: The complete dataset fetched from the API.

    Raises:
        requests.RequestException: If there's an error in making the API request.
    """
    offset = 0
    all_data = []
    total_rows_fetched = 0
    
    logger.info(f"Starting data retrieval from {url}")
    logger.info(f"Chunk size: {chunk_size}, Max rows: {max_rows if max_rows else 'No limit'}")
    
    while True:
        # Construct the URL with offset and limit parameters
        paginated_url = f"{url}?$offset={offset}&$limit={chunk_size}"

        # Make the API request
        response = requests.get(paginated_url)
        
        # Check if the request was successful
        if response.status_code == 200:
            # Convert the response content to a pandas DataFrame
            chunk = pd.json_normalize(response.json())
            
            # If the chunk is empty, we've reached the end of the data
            if chunk.empty:
                logger.info("Received empty chunk. Ending pagination.")
                break
            
            # Append the chunk to our list of DataFrames
            all_data.append(chunk)
            logger.info("Data size: "+ str(len(all_data)))
            # Update total rows fetched
            total_rows_fetched += len(chunk)
            logger.info(f"Fetched {len(chunk)} rows. Total rows so far: {total_rows_fetched}")
            # Increment the offset for the next request
            offset += chunk_size
            
            # If we've reached the maximum number of rows, stop
            if max_rows and total_rows_fetched >= max_rows:
                logger.info(f"Reached or exceeded max rows ({max_rows}). Stopping pagination.")
                break
        else:
            logger.error(f"Error fetching data: HTTP {response.status_code}")
            break
    
    # Combine all chunks into a single DataFrame
    logger.info("Combining all fetched data into a single DataFrame")
    full_dataset = pd.concat(all_data, ignore_index=True)
    
    # If max_rows was specified, trim the dataset
    if max_rows and len(full_dataset) > max_rows:
        logger.info(f"Trimming dataset to {max_rows} rows")
        full_dataset = full_dataset.head(max_rows)
    
    logger.info(f"Final dataset size: {len(full_dataset)} rows")
    
    # Save the dataset as a CSV file
    current_date = datetime.now().strftime("%Y%m%d")
    dirname = ''
    filename = f"data_{current_date}.csv"
    full_dataset.to_csv(dirname + filename, index=False)
    logger.info(f"Dataset saved to {filename}")    
    
    return full_dataset

In [32]:
url = "https://analisi.transparenciacatalunya.cat/resource/nzvn-apee.json"
df = load_json_from_api(url, chunk_size=10, max_rows=100)
logger.info(f"Loaded {len(df)} rows of data")
df.head(10)

INFO:__main__:Starting data retrieval from https://analisi.transparenciacatalunya.cat/resource/nzvn-apee.json
INFO:__main__:Chunk size: 10, Max rows: 100
INFO:__main__:Data size: 1
INFO:__main__:Fetched 10 rows. Total rows so far: 10
INFO:__main__:Data size: 2
INFO:__main__:Fetched 10 rows. Total rows so far: 20
INFO:__main__:Data size: 3
INFO:__main__:Fetched 10 rows. Total rows so far: 30
INFO:__main__:Data size: 4
INFO:__main__:Fetched 10 rows. Total rows so far: 40
INFO:__main__:Data size: 5
INFO:__main__:Fetched 10 rows. Total rows so far: 50
INFO:__main__:Data size: 6
INFO:__main__:Fetched 10 rows. Total rows so far: 60
INFO:__main__:Data size: 7
INFO:__main__:Fetched 10 rows. Total rows so far: 70
INFO:__main__:Data size: 8
INFO:__main__:Fetched 10 rows. Total rows so far: 80
INFO:__main__:Data size: 9
INFO:__main__:Fetched 10 rows. Total rows so far: 90
INFO:__main__:Data size: 10
INFO:__main__:Fetched 10 rows. Total rows so far: 100
INFO:__main__:Reached or exceeded max rows (

,id,codi_estacio,codi_variable,data_lectura,valor_lectura,codi_estat,codi_base,data_extrem
0,VC462405221230,VC,46,2022-05-24T12:30:00.000,1,V,SH,NaN
1,VC472405221230,VC,47,2022-05-24T12:30:00.000,215,V,SH,NaN
2,VC562405221230,VC,56,2022-05-24T12:30:00.000,4.9,V,SH,2022-05-24T12:56:00.000
3,D1511712201930,D1,51,2020-12-17T19:30:00.000,253,V,SH,NaN
4,XG032109201330,XG,3,2020-09-21T13:30:00.000,66,V,SH,2020-09-21T13:50:00.000
5,XG302109201330,XG,30,2020-09-21T13:30:00.000,0.9,V,SH,NaN
6,XG312109201330,XG,31,2020-09-21T13:30:00.000,83,V,SH,NaN
7,XG322109201330,XG,32,2020-09-21T13:30:00.000,23.3,V,SH,NaN
8,XG332109201330,XG,33,2020-09-21T13:30:00.000,64,V,SH,NaN
9,XG342109201330,XG,34,2020-09-21T13:30:00.000,1002.2,V,SH,NaN


In [31]:
from sodapy import Socrata

client = Socrata("analisi.transparenciacatalunya.cat", None)

#results = client.get("nzvn-apee", limit=10, order="codi_variable ASC")
results = client.get("nzvn-apee", limit=10, order="valor_lectura ASC", where="codi_variable=32")

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
results_df.head(10)

HTTPError: 400 Client Error: Bad Request.
	Query coordinator error: query.soql.type-mismatch; Type mismatch for op$=, is number; position: Map(row -> 1, column -> 145, line -> "SELECT `id`, `codi_estacio`, `codi_variable`, `data_lectura`, `data_extrem`, `valor_lectura`, `codi_estat`, `codi_base` WHERE `codi_variable` = 32 ORDER BY `valor_lectura` ASC NULL LAST LIMIT 10\n                                                                                                                                                ^")

### Convert data from object to float


In [7]:
# Create a DataFrame with numbers as strings with commas
data = {
    'A': ['1,00', '2,50', '3,750'],
    'B': ['4,20', '5,10', '6,850'],
    'C': ['7,00', '8,15', '9,900']
}

df = pd.DataFrame(data)
df.dtypes

A    object
B    object
C    object
dtype: object

In [8]:
# Convert the numbers to floats by removing commas
df['A'] = df['A'].replace(',', '.', regex=True).astype(float)
df.dtypes

A    float64
B     object
C     object
dtype: object

In [9]:
# Convert the numbers to floats by removing commas
df['B'] = df['B'].apply(lambda x: float(x.replace(',', '.')))
df.dtypes

A    float64
B    float64
C     object
dtype: object

In [10]:
df['C'] = pd.to_numeric(df['C'].str.replace(',', '.'))
df.dtypes

A    float64
B    float64
C    float64
dtype: object

In [11]:
df

,A,B,C
0,1.00,4.20,7.00
1,2.50,5.10,8.15
2,3.75,6.85,9.90
